本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [1]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
import re

# 1. 提取正文部分的文本特征；
data_path = './trec06c-utf8/data'
data_cut_path = './trec06c-utf8/data_cut'
label_file = './trec06c-utf8/label/index'
print_email_count = 3
email_addr_simple_mod = re.compile('<(.+@.+)>')
dataset = []
for label_line in open(label_file, mode='r', encoding='utf8'):
    row = []
    labels = label_line.strip().split(' ')
    # 是否垃圾邮件
    is_spam = 1 if labels[0] == 'spam' else 0
    row.append(is_spam)
    # 邮件路径
    email_data_cut_path = data_cut_path + labels[1][len('../data'):]
    row.append(email_data_cut_path)
    # 邮件正文
    email_body_start = False
    email_body_str = ''
    email_header_info = ''
    email_addr_list = []
    for email_line in open(email_data_cut_path, mode='r', encoding='utf8'):
        email_line_trim = email_line.strip()
        if email_line_trim != '':
            if email_body_start:
                # 邮件体组成一行，以空格分隔
                email_body_str += (email_line_trim + ' ')
            else:
                # 邮件头（后期处理）
                if len(email_addr_list) == 0:
                    if email_line_trim.startswith('From:') or email_line_trim.startswith('Reply-To:'):
                        email_addr_list = email_addr_simple_mod.findall(email_line_trim)
                #continue
        else:
            # 分隔行/邮件体中的空行
            email_body_start = True
    # 过滤掉非中文字符
    email_body_str = re.sub(r'[^\u4e00-\u9fa5 ]', '', email_body_str)
    row.append(email_body_str)
    if print_email_count >= 0:
        print(email_body_str)
        print_email_count -= 1
    # 邮件头信息
    email_header_info = email_addr_list[0] if len(email_addr_list) != 0 else ''
    row.append(email_header_info)
    dataset.append(row)

 课   程   背   景  每 一位 管理 和 技术人员 都 清楚 地 懂得  单纯 从 技术 角度 衡量 为 合算 的 方案  也许 却是 一个 财务 陷阱  表面 赢利 而 暗地里 亏损  使经 营者 无法 接受  如何 将 技术手段 与 财务 运作 相结合  使 每位 管理 和 技术人员 都 从 本 课程 通过 沙盘 模拟 和 案例 分析  使 企业 各级 管理 和 技术人员 掌握 财务管理 知识  利用 财务 信息 改进 管理决策  实现 管理 效益 最大化  通过 学习 本 课程  您 将     对 会计 与 财务管理 有 基本 了解  提高 日常 管理 活动 的 财务 可行性     通过 分析 关键 业绩 指标  形成 战略规划 与 全面 预算     突出 企业 管理 的 重心  形成 管理 的 系统性   课   程   大   纲  一  财务 工作 内容 及 作用   财务 专家 的 思维 模式   财务 工作 的 基本 内容   管理者 如何 利用 财务 进行 管理 和 决策 二  如何 阅读 和 分析 财务报表   会计报表 的 构成   损益表 的 阅读 与 分析   资产 负债表 的 阅读 与 分析   资金 流量 和 现金流量 表 的 阅读 与 分析   会计报表 之间 的 关系  案例 分析  读 报表  判断 企业 业绩 水平 三  如何 运用 财务 手段 进行 成本 控制   产品成本 的 概念 和 构成       本  浚利  分析 与 运用   标准 成本 制度 在 成本 控制 中 的 作用   如何 运用 目标 成本法 控制 产品成本  保证 利润 水平   如何 运用  作业 成本法 进行 管理 分析  实施 精细 成本 管理   如何 针对 沉没 成本 和 机会成本 进行 正确 决策   如何 改善 采购  生产 等 环节 的 运作 以 改良 企业 的 整体 财务状况  综合 案例 分析   管理 和 技术 方案 的 可行性 分析   新 产品开发 中 的 财务 可行性 分析   产品 增产  减产 时 的 财务 可行性 分析   生产 设备 改造 与 更新 的 决策分析   投资 项目 的 现金流 分析   投资 项目 评价 方法  净 现值 法分析  资金 时间 价值 分析   综合 案例 演练 五  公司

In [2]:
# 2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
train_set_ratio = 0.8
train_row_num = int(len(dataset) * train_set_ratio)
X_train = []
y_train = []
X_test = []
y_test = []
for index in range(len(dataset)):
    if index < train_row_num:
        X_train.append(dataset[index][2])
        y_train.append(dataset[index][0])
    else:
        X_test.append(dataset[index][2])
        y_test.append(dataset[index][0])

In [3]:
# 3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
from sklearn.metrics import classification_report
# 加载停用词
stop_words = []
for line in open('./cn_stopwords.txt', mode='r', encoding='utf8'):
    stop_words.append(line.strip())

vectorizer = CountVectorizer(stop_words=stop_words)
X_train_count = vectorizer.fit_transform(X_train)
print('特征总数：', len(vectorizer.get_feature_names_out()))
print('前50个特征：', vectorizer.get_feature_names_out()[0:50])
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train_count, y_train)
X_test_count = vectorizer.transform(X_test)
y_test_predict = mnb_clf.predict(X_test_count)
print(classification_report(y_test, y_test_predict, target_names=['ham','spam'], digits=4))

特征总数： 126551
前50个特征： ['一一' '一一列举' '一一对应' '一丁点' '一丁点儿' '一万' '一万七千' '一万三千' '一万两千' '一万个' '一万二'
 '一万二千' '一万五' '一万五千个' '一万余' '一万倍' '一万元' '一万八千' '一万八千余' '一万六千多' '一万卷' '一万名'
 '一万四千' '一万块' '一万多' '一万多个' '一万多元' '一万多块' '一万左右' '一万年' '一万条' '一万次' '一万步'
 '一万遍' '一上' '一上午' '一上量' '一下' '一下一下' '一下下' '一下半' '一下头' '一下子' '一下子全部' '一下子发'
 '一下子成' '一下子把' '一下手' '一不做' '一不小心']
              precision    recall  f1-score   support

         ham     0.9335    0.9673    0.9501      4498
        spam     0.9822    0.9632    0.9726      8426

    accuracy                         0.9646     12924
   macro avg     0.9579    0.9653    0.9614     12924
weighted avg     0.9653    0.9646    0.9648     12924



In [4]:
# 4. 对比特征数目（词表大小）对模型效果的影响；
vectorizer_maxfeat = CountVectorizer(stop_words=stop_words, max_features=20000)
X_train_count = vectorizer_maxfeat.fit_transform(X_train)
print('特征总数：', len(vectorizer_maxfeat.get_feature_names_out()))
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train_count, y_train)
X_test_count = vectorizer_maxfeat.transform(X_test)
y_test_predict = mnb_clf.predict(X_test_count)
print(classification_report(y_test, y_test_predict, target_names=['ham','spam'], digits=4))
print('特征数目由126551降到20000，准确率Accuracy从0.9646降到0.9622（只是测试）')

特征总数： 20000
              precision    recall  f1-score   support

         ham     0.9282    0.9660    0.9467      4498
        spam     0.9814    0.9601    0.9707      8426

    accuracy                         0.9622     12924
   macro avg     0.9548    0.9631    0.9587     12924
weighted avg     0.9629    0.9622    0.9623     12924

特征数目由126551降到20000，准确率Accuracy从0.9646降到0.9622（只是测试）


In [5]:
# 扩展要求：
# 1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
# 提取发件人邮箱地址作为特征之一
train_set_ratio = 0.8
train_row_num = int(len(dataset) * train_set_ratio)
X_train = []
y_train = []
X_test = []
y_test = []
for index in range(len(dataset)):
    if index < train_row_num:
        X_train.append(dataset[index][2] + dataset[index][3])
        y_train.append(dataset[index][0])
    else:
        X_test.append(dataset[index][2] + dataset[index][3])
        y_test.append(dataset[index][0])

vectorizer = CountVectorizer(stop_words=stop_words)
X_train_count = vectorizer.fit_transform(X_train)
print('特征总数：', len(vectorizer.get_feature_names_out()))
X_test_count = vectorizer.transform(X_test)
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train_count, y_train)
y_test_predict = mnb_clf.predict(X_test_count)
print('Classification Report for MultinomialNB')
print(classification_report(y_test, y_test_predict, target_names=['ham','spam'], digits=4))

# 2. 尝试自行实现朴素贝叶斯算法细节；
# 暂未实现
# 根据训练集计算 0-ham；1-spam对应数据集中各个特征的频次及概率，再针对测试集每个样本分别计算0-ham；1-spam下的联合概率，取最大对应的label

# 3. 尝试对比不同的概率计算方法。
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train_count, y_train)
y_test_predict = bnb_clf.predict(X_test_count)
print('Classification Report for BernoulliNB')
print(classification_report(y_test, y_test_predict, target_names=['ham','spam'], digits=4))
cnb_clf = ComplementNB()
cnb_clf.fit(X_train_count, y_train)
y_test_predict = cnb_clf.predict(X_test_count)
print('Classification Report for ComplementNB')
print(classification_report(y_test, y_test_predict, target_names=['ham','spam'], digits=4))

特征总数： 130715
Classification Report for MultinomialNB
              precision    recall  f1-score   support

         ham     0.9449    0.9724    0.9585      4498
        spam     0.9851    0.9697    0.9773      8426

    accuracy                         0.9707     12924
   macro avg     0.9650    0.9711    0.9679     12924
weighted avg     0.9711    0.9707    0.9708     12924

Classification Report for BernoulliNB
              precision    recall  f1-score   support

         ham     0.8184    0.9758    0.8902      4498
        spam     0.9856    0.8844    0.9323      8426

    accuracy                         0.9162     12924
   macro avg     0.9020    0.9301    0.9112     12924
weighted avg     0.9274    0.9162    0.9176     12924

Classification Report for ComplementNB
              precision    recall  f1-score   support

         ham     0.9231    0.9740    0.9479      4498
        spam     0.9857    0.9567    0.9710      8426

    accuracy                         0.9627     1292